In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import csv
from PIL import Image
import time
import cv2
from torch.utils.data import Dataset
from torchvision import transforms
from tensorflow.keras.preprocessing.image import img_to_array


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# image_dir = "/home/selloh/externalProjects/TransferLearning/datasets/extractedFrames"
design_dir = "/home/selloh/externalProjects/TransferLearning/datasets/DREYEVE_DATA/dreyeve_design.txt"
subsequence_dir = "/home/selloh/externalProjects/TransferLearning/datasets/DREYEVE_DATA/subsequences.txt"
speed_data_dir = "/home/selloh/externalProjects/TransferLearning/Dreyeve/data/speed_data.csv"
gaze_data_dir = "/home/selloh/externalProjects/TransferLearning/Dreyeve/data/gaze_data.csv"
# image_data = "/home/selloh/externalProjects/TransferLearning/datasets/DREYEVE_DATA"
# extracted_frames_dir = "/home/selloh/externalProjects/TransferLearning/datasets/extractedFrames"


In [3]:
# opening directories
design = pd.read_csv(design_dir, sep="\t")
subsequence = pd.read_csv(subsequence_dir, sep="\t")
speed_data = pd.read_csv(speed_data_dir)
gaze_data = pd.read_csv(gaze_data_dir)

In [4]:
design
design = design.T.reset_index().T.reset_index(drop=True)
design
design.columns = ['file_number', 'time', 'weather', 'location', 'D', 'train/test']


In [5]:
design

,file_number,time,weather,location,D,train/test
0,01,Evening,Sunny,Countryside,D8,Training Set
1,2,Morning,Cloudy,Highway,D2,Training Set
2,3,Evening,Sunny,Highway,D3,Training Set
3,4,Night,Sunny,Downtown,D2,Training Set
4,5,Morning,Cloudy,Countryside,D7,Training Set
...,...,...,...,...,...,...
69,70,Morning,Rainy,Downtown,D3,Test Set
70,71,Night,Cloudy,Highway,D6,Test Set
71,72,Evening,Cloudy,Downtown,D2,Test Set
72,73,Night,Sunny,Countryside,D7,Test Set


In [6]:
subsequence = subsequence.T.reset_index().T.reset_index(drop=True)
subsequence.columns = ['file_number', 'start_frame', 'end_frame', 'attention_type']
subsequence


,file_number,start_frame,end_frame,attention_type
0,1,1.1,27,k
1,1,36,72,k
2,1,483,489,k
3,1,509,684,e
4,1,685,686,e
...,...,...,...,...
3550,74,6103,6117,k
3551,74,6192,6195,k
3552,74,7103,7118,k
3553,74,7331,7351,i


In [7]:
gaze_data = gaze_data.iloc[1: , :]
gaze_data = gaze_data.reset_index(drop=True)
gaze_data


,file_number,frame_etg,frame_gar,X,Y,event_type,code
0,1,0,0,313.81,256.40,Fixation,911066603
1,1,0,0,313.56,258.52,Saccade,911083143
2,1,1,1,292.11,259.90,Saccade,911099864
3,1,1,1,185.82,306.49,Saccade,911116251
4,1,2,2,143.25,322.21,Saccade,911133030
...,...,...,...,...,...,...,...
1301245,74,8999,7499,405.71,207.12,Saccade,709796963
1301246,74,8999,7499,495.73,179.57,Saccade,709813635
1301247,74,9000,7500,589.77,152.12,Saccade,709830213
1301248,74,9000,7500,690.76,123.23,Fixation,709847356


In [8]:
gaze_data['frame_gar'] = pd.to_numeric(gaze_data['frame_gar'], errors='coerce')
gaze_data['timeinsec'] = gaze_data['frame_gar']*1/30

In [9]:
speed_data

,file_number,frame_num,speed,course,lat,lon
0,1,1,34,348.0,0.0,0.0
1,1,2,35,347.0,0.0,0.0
2,1,3,35,347.0,0.0,0.0
3,1,4,35,346.0,0.0,0.0
4,1,5,35,346.0,0.0,0.0
...,...,...,...,...,...,...
554995,74,7496,0,302.0,0.0,0.0
554996,74,7497,0,302.0,0.0,0.0
554997,74,7498,0,302.0,0.0,0.0
554998,74,7499,0,302.0,0.0,0.0


# Combining All Data

In [26]:
subsequence

,file_number,start_frame,end_frame,attention_type
0,1,1.1,27,k
1,1,36,72,k
2,1,483,489,k
3,1,509,684,e
4,1,685,686,e
...,...,...,...,...
3550,74,6103,6117,k
3551,74,6192,6195,k
3552,74,7103,7118,k
3553,74,7331,7351,i


In [27]:
gaze_data

,file_number,frame_etg,frame_gar,X,Y,event_type,code,timeinsec
0,1,0,0.0,313.81,256.40,Fixation,911066603,0.000000
1,1,0,0.0,313.56,258.52,Saccade,911083143,0.000000
2,1,1,1.0,292.11,259.90,Saccade,911099864,0.033333
3,1,1,1.0,185.82,306.49,Saccade,911116251,0.033333
4,1,2,2.0,143.25,322.21,Saccade,911133030,0.066667
...,...,...,...,...,...,...,...,...
1301245,74,8999,7499.0,405.71,207.12,Saccade,709796963,249.966667
1301246,74,8999,7499.0,495.73,179.57,Saccade,709813635,249.966667
1301247,74,9000,7500.0,589.77,152.12,Saccade,709830213,250.000000
1301248,74,9000,7500.0,690.76,123.23,Fixation,709847356,250.000000


In [28]:
speed_data

,file_number,frame_num,speed,course,lat,lon
0,1,1,34,348.0,0.0,0.0
1,1,2,35,347.0,0.0,0.0
2,1,3,35,347.0,0.0,0.0
3,1,4,35,346.0,0.0,0.0
4,1,5,35,346.0,0.0,0.0
...,...,...,...,...,...,...
554995,74,7496,0,302.0,0.0,0.0
554996,74,7497,0,302.0,0.0,0.0
554997,74,7498,0,302.0,0.0,0.0
554998,74,7499,0,302.0,0.0,0.0


In [29]:
design

,file_number,time,weather,location,D,train/test
0,01,Evening,Sunny,Countryside,D8,Training Set
1,2,Morning,Cloudy,Highway,D2,Training Set
2,3,Evening,Sunny,Highway,D3,Training Set
3,4,Night,Sunny,Downtown,D2,Training Set
4,5,Morning,Cloudy,Countryside,D7,Training Set
...,...,...,...,...,...,...
69,70,Morning,Rainy,Downtown,D3,Test Set
70,71,Night,Cloudy,Highway,D6,Test Set
71,72,Evening,Cloudy,Downtown,D2,Test Set
72,73,Night,Sunny,Countryside,D7,Test Set


In [30]:
design.iloc[0, 0] = '1'


In [31]:
design

,file_number,time,weather,location,D,train/test
0,1,Evening,Sunny,Countryside,D8,Training Set
1,2,Morning,Cloudy,Highway,D2,Training Set
2,3,Evening,Sunny,Highway,D3,Training Set
3,4,Night,Sunny,Downtown,D2,Training Set
4,5,Morning,Cloudy,Countryside,D7,Training Set
...,...,...,...,...,...,...
69,70,Morning,Rainy,Downtown,D3,Test Set
70,71,Night,Cloudy,Highway,D6,Test Set
71,72,Evening,Cloudy,Downtown,D2,Test Set
72,73,Night,Sunny,Countryside,D7,Test Set


In [33]:
import pandas as pd
design['file_number'] = design['file_number'].astype(int)
gaze_data['file_number'] = gaze_data['file_number'].astype(int)
merged_df = pd.merge(gaze_data, design, on='file_number')
merged_df

,file_number,frame_etg,frame_gar,X,Y,event_type,code,timeinsec,time,weather,location,D,train/test
0,1,0,0.0,313.81,256.40,Fixation,911066603,0.000000,Evening,Sunny,Countryside,D8,Training Set
1,1,0,0.0,313.56,258.52,Saccade,911083143,0.000000,Evening,Sunny,Countryside,D8,Training Set
2,1,1,1.0,292.11,259.90,Saccade,911099864,0.033333,Evening,Sunny,Countryside,D8,Training Set
3,1,1,1.0,185.82,306.49,Saccade,911116251,0.033333,Evening,Sunny,Countryside,D8,Training Set
4,1,2,2.0,143.25,322.21,Saccade,911133030,0.066667,Evening,Sunny,Countryside,D8,Training Set
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301245,74,8999,7499.0,405.71,207.12,Saccade,709796963,249.966667,Morning,Rainy,Downtown,D4,Test Set
1301246,74,8999,7499.0,495.73,179.57,Saccade,709813635,249.966667,Morning,Rainy,Downtown,D4,Test Set
1301247,74,9000,7500.0,589.77,152.12,Saccade,709830213,250.000000,Morning,Rainy,Downtown,D4,Test Set
1301248,74,9000,7500.0,690.76,123.23,Fixation,709847356,250.000000,Morning,Rainy,Downtown,D4,Test Set


In [34]:

speed_data = speed_data.rename(columns={'frame_num': 'frame_gar'})
speed_data



,file_number,frame_gar,speed,course,lat,lon
0,1,1,34,348.0,0.0,0.0
1,1,2,35,347.0,0.0,0.0
2,1,3,35,347.0,0.0,0.0
3,1,4,35,346.0,0.0,0.0
4,1,5,35,346.0,0.0,0.0
...,...,...,...,...,...,...
554995,74,7496,0,302.0,0.0,0.0
554996,74,7497,0,302.0,0.0,0.0
554997,74,7498,0,302.0,0.0,0.0
554998,74,7499,0,302.0,0.0,0.0


In [35]:
import pandas as pd
merged_data = pd.merge(merged_df, speed_data, on=['frame_gar', 'file_number'], how='left')
merged_data


,file_number,frame_etg,frame_gar,X,Y,event_type,code,timeinsec,time,weather,location,D,train/test,speed,course,lat,lon
0,1,0,0.0,313.81,256.40,Fixation,911066603,0.000000,Evening,Sunny,Countryside,D8,Training Set,NaN,NaN,NaN,NaN
1,1,0,0.0,313.56,258.52,Saccade,911083143,0.000000,Evening,Sunny,Countryside,D8,Training Set,NaN,NaN,NaN,NaN
2,1,1,1.0,292.11,259.90,Saccade,911099864,0.033333,Evening,Sunny,Countryside,D8,Training Set,34.0,348.0,0.0,0.0
3,1,1,1.0,185.82,306.49,Saccade,911116251,0.033333,Evening,Sunny,Countryside,D8,Training Set,34.0,348.0,0.0,0.0
4,1,2,2.0,143.25,322.21,Saccade,911133030,0.066667,Evening,Sunny,Countryside,D8,Training Set,35.0,347.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301245,74,8999,7499.0,405.71,207.12,Saccade,709796963,249.966667,Morning,Rainy,Downtown,D4,Test Set,0.0,302.0,0.0,0.0
1301246,74,8999,7499.0,495.73,179.57,Saccade,709813635,249.966667,Morning,Rainy,Downtown,D4,Test Set,0.0,302.0,0.0,0.0
1301247,74,9000,7500.0,589.77,152.12,Saccade,709830213,250.000000,Morning,Rainy,Downtown,D4,Test Set,0.0,302.0,0.0,0.0
1301248,74,9000,7500.0,690.76,123.23,Fixation,709847356,250.000000,Morning,Rainy,Downtown,D4,Test Set,0.0,302.0,0.0,0.0


In [36]:
subsequence['start_frame'] = subsequence['start_frame'].astype(float).round().astype(int)
subsequence['end_frame'] = subsequence['end_frame'].astype(float).round().astype(int)


In [37]:
subsequence['start_frame'] = pd.to_numeric(subsequence['start_frame'])
subsequence['end_frame'] = pd.to_numeric(subsequence['end_frame'])

def assign_attention(df, subsequence):

    for index, row in subsequence.iterrows():
        start = row['start_frame']
        end = row['end_frame']
        attention_type = row['attention_type']
        
        # Update the 'attention_type' column based on the frame range
        df.loc[(df['frame_gar'] >= start) & (df['frame_gar'] <= end), 'attention_type'] = attention_type

    return df


In [38]:
def assign_timestamp(df):
    FPS = 30

    for index, row in df.iterrows():
        row['timeinsec'] = row['frame_gar']*1/FPS

    return df

merged_data = merged_df.groupby('file_number').apply(assign_timestamp)


/tmp/ipykernel_97120/1691273844.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_data = merged_df.groupby('file_number').apply(assign_timestamp)


In [39]:
merged_data = assign_attention(merged_data, subsequence)

In [40]:
merged_data 

file_number frame_etg  frame_gar       X       Y  \
file_number                                                             
1           0                  1         0        0.0  313.81  256.40   
            1                  1         0        0.0  313.56  258.52   
            2                  1         1        1.0  292.11  259.90   
            3                  1         1        1.0  185.82  306.49   
            4                  1         2        2.0  143.25  322.21   
...                          ...       ...        ...     ...     ...   
74          1301245           74      8999     7499.0  405.71  207.12   
            1301246           74      8999     7499.0  495.73  179.57   
            1301247           74      9000     7500.0  589.77  152.12   
            1301248           74      9000     7500.0  690.76  123.23   
            1301249           74      9000     7500.0  589.77  152.12   

                    event_type       code   timeinsec     time weather  \
file_number                                                              
1           0         Fixation  911066603    0.000000  Evening   Sunny   
            1          Saccade  911083143    0.000000  Evening   Sunny   
            2          Saccade  911099864    0.033333  Evening   Sunny   
            3          Saccade  911116251    0.033333  Evening   Sunny   
            4          Saccade  911133030    0.066667  Evening   Sunny   
...                        ...        ...         ...      ...     ...   
74          1301245    Saccade  709796963  249.966667  Morning   Rainy   
            1301246    Saccade  709813635  249.966667  Morning   Rainy   
            1301247    Saccade  709830213  250.000000  Morning   Rainy   
            1301248   Fixation  709847356  250.000000  Morning   Rainy   
            1301249   Fixation  709863411  250.000000  Morning   Rainy   

                        location   D    train/test attention_type  
file_number                                                        
1           0        Countryside  D8  Training Set            NaN  
            1        Countryside  D8  Training Set            NaN  
            2        Countryside  D8  Training Set              i  
            3        Countryside  D8  Training Set              i  
            4        Countryside  D8  Training Set              i  
...                          ...  ..           ...            ...  
74          1301245     Downtown  D4      Test Set              i  
            1301246     Downtown  D4      Test Set              i  
            1301247     Downtown  D4      Test Set              i  
            1301248     Downtown  D4      Test Set              i  
            1301249     Downtown  D4      Test Set              i  

[1301250 rows x 14 columns]

In [41]:
merged_data

file_number frame_etg  frame_gar       X       Y  \
file_number                                                             
1           0                  1         0        0.0  313.81  256.40   
            1                  1         0        0.0  313.56  258.52   
            2                  1         1        1.0  292.11  259.90   
            3                  1         1        1.0  185.82  306.49   
            4                  1         2        2.0  143.25  322.21   
...                          ...       ...        ...     ...     ...   
74          1301245           74      8999     7499.0  405.71  207.12   
            1301246           74      8999     7499.0  495.73  179.57   
            1301247           74      9000     7500.0  589.77  152.12   
            1301248           74      9000     7500.0  690.76  123.23   
            1301249           74      9000     7500.0  589.77  152.12   

                    event_type       code   timeinsec     time weather  \
file_number                                                              
1           0         Fixation  911066603    0.000000  Evening   Sunny   
            1          Saccade  911083143    0.000000  Evening   Sunny   
            2          Saccade  911099864    0.033333  Evening   Sunny   
            3          Saccade  911116251    0.033333  Evening   Sunny   
            4          Saccade  911133030    0.066667  Evening   Sunny   
...                        ...        ...         ...      ...     ...   
74          1301245    Saccade  709796963  249.966667  Morning   Rainy   
            1301246    Saccade  709813635  249.966667  Morning   Rainy   
            1301247    Saccade  709830213  250.000000  Morning   Rainy   
            1301248   Fixation  709847356  250.000000  Morning   Rainy   
            1301249   Fixation  709863411  250.000000  Morning   Rainy   

                        location   D    train/test attention_type  
file_number                                                        
1           0        Countryside  D8  Training Set            NaN  
            1        Countryside  D8  Training Set            NaN  
            2        Countryside  D8  Training Set              i  
            3        Countryside  D8  Training Set              i  
            4        Countryside  D8  Training Set              i  
...                          ...  ..           ...            ...  
74          1301245     Downtown  D4      Test Set              i  
            1301246     Downtown  D4      Test Set              i  
            1301247     Downtown  D4      Test Set              i  
            1301248     Downtown  D4      Test Set              i  
            1301249     Downtown  D4      Test Set              i  

[1301250 rows x 14 columns]

In [42]:
merged_data = merged_data.reset_index(drop=True)
merged_data

,file_number,frame_etg,frame_gar,X,Y,event_type,code,timeinsec,time,weather,location,D,train/test,attention_type
0,1,0,0.0,313.81,256.40,Fixation,911066603,0.000000,Evening,Sunny,Countryside,D8,Training Set,NaN
1,1,0,0.0,313.56,258.52,Saccade,911083143,0.000000,Evening,Sunny,Countryside,D8,Training Set,NaN
2,1,1,1.0,292.11,259.90,Saccade,911099864,0.033333,Evening,Sunny,Countryside,D8,Training Set,i
3,1,1,1.0,185.82,306.49,Saccade,911116251,0.033333,Evening,Sunny,Countryside,D8,Training Set,i
4,1,2,2.0,143.25,322.21,Saccade,911133030,0.066667,Evening,Sunny,Countryside,D8,Training Set,i
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301245,74,8999,7499.0,405.71,207.12,Saccade,709796963,249.966667,Morning,Rainy,Downtown,D4,Test Set,i
1301246,74,8999,7499.0,495.73,179.57,Saccade,709813635,249.966667,Morning,Rainy,Downtown,D4,Test Set,i
1301247,74,9000,7500.0,589.77,152.12,Saccade,709830213,250.000000,Morning,Rainy,Downtown,D4,Test Set,i
1301248,74,9000,7500.0,690.76,123.23,Fixation,709847356,250.000000,Morning,Rainy,Downtown,D4,Test Set,i


In [43]:
merged_data = merged_data.drop(columns=['file_number','frame_etg', 'frame_gar','code', 'D', 'train/test'])

In [44]:
merged_data.shape

(1301250, 8)

In [45]:
data_without_weather = merged_data.drop(columns=['time', 'weather', 'location'])

In [46]:
merged_data

,X,Y,event_type,timeinsec,time,weather,location,attention_type
0,313.81,256.40,Fixation,0.000000,Evening,Sunny,Countryside,NaN
1,313.56,258.52,Saccade,0.000000,Evening,Sunny,Countryside,NaN
2,292.11,259.90,Saccade,0.033333,Evening,Sunny,Countryside,i
3,185.82,306.49,Saccade,0.033333,Evening,Sunny,Countryside,i
4,143.25,322.21,Saccade,0.066667,Evening,Sunny,Countryside,i
...,...,...,...,...,...,...,...,...
1301245,405.71,207.12,Saccade,249.966667,Morning,Rainy,Downtown,i
1301246,495.73,179.57,Saccade,249.966667,Morning,Rainy,Downtown,i
1301247,589.77,152.12,Saccade,250.000000,Morning,Rainy,Downtown,i
1301248,690.76,123.23,Fixation,250.000000,Morning,Rainy,Downtown,i


In [47]:
merged_data = merged_data.reset_index(drop=True)
merged_data

,X,Y,event_type,timeinsec,time,weather,location,attention_type
0,313.81,256.40,Fixation,0.000000,Evening,Sunny,Countryside,NaN
1,313.56,258.52,Saccade,0.000000,Evening,Sunny,Countryside,NaN
2,292.11,259.90,Saccade,0.033333,Evening,Sunny,Countryside,i
3,185.82,306.49,Saccade,0.033333,Evening,Sunny,Countryside,i
4,143.25,322.21,Saccade,0.066667,Evening,Sunny,Countryside,i
...,...,...,...,...,...,...,...,...
1301245,405.71,207.12,Saccade,249.966667,Morning,Rainy,Downtown,i
1301246,495.73,179.57,Saccade,249.966667,Morning,Rainy,Downtown,i
1301247,589.77,152.12,Saccade,250.000000,Morning,Rainy,Downtown,i
1301248,690.76,123.23,Fixation,250.000000,Morning,Rainy,Downtown,i


In [48]:
merged_data['X'] = pd.to_numeric(merged_data['X'], errors='coerce')
merged_data['Y'] = pd.to_numeric(merged_data['Y'], errors='coerce')

In [49]:
merged_data = merged_data[merged_data['attention_type'] != 'e']

In [50]:
merged_data = merged_data[~merged_data['event_type'].isin(['event_type', '-'])]

In [63]:
merged_data.columns

Index(['X', 'Y', 'event_type', 'timeinsec', 'time', 'weather', 'location',
       'attention_type'],
      dtype='object')

In [52]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Feature Importance Analysis

In [53]:
X = merged_data.drop(columns=['attention_type'])
y = merged_data['attention_type']
X_encoded  = pd.get_dummies(X, columns=['event_type', 'time', 'weather', 'location'], dtype=int)
y_encoded = pd.get_dummies(y)

In [54]:
X_encoded

,X,Y,timeinsec,event_type_Blink,event_type_Fixation,event_type_Saccade,time_Evening,time_Morning,time_Night,weather_Cloudy,weather_Rainy,weather_Sunny,location_Countryside,location_Downtown,location_Highway
0,313.81,256.40,0.000000,0,1,0,1,0,0,0,0,1,1,0,0
1,313.56,258.52,0.000000,0,0,1,1,0,0,0,0,1,1,0,0
2,292.11,259.90,0.033333,0,0,1,1,0,0,0,0,1,1,0,0
3,185.82,306.49,0.033333,0,0,1,1,0,0,0,0,1,1,0,0
4,143.25,322.21,0.066667,0,0,1,1,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301245,405.71,207.12,249.966667,0,0,1,0,1,0,0,1,0,0,1,0
1301246,495.73,179.57,249.966667,0,0,1,0,1,0,0,1,0,0,1,0
1301247,589.77,152.12,250.000000,0,0,1,0,1,0,0,1,0,0,1,0
1301248,690.76,123.23,250.000000,0,1,0,0,1,0,0,1,0,0,1,0


In [55]:
x_train, x_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2)

In [56]:
y_train

,i,k,u
658339,True,False,False
360008,False,True,False
185125,False,True,False
898721,False,True,False
853910,False,True,False
...,...,...,...
900154,False,True,False
1048211,False,True,False
1217220,False,True,False
655583,False,True,False


In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [58]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [59]:
x_train_scaled.shape

(759230, 15)

In [60]:
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
model.fit(x_train_scaled, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [61]:
import joblib

In [62]:
dfgdfg

NameError: name 'dfgdfg' is not defined

In [64]:
joblib.dump(model, 'random_forest_model_second_try.pkl')

['random_forest_model_second_try.pkl']

In [65]:
y_pred = model.predict(x_test_scaled)

In [66]:
print(y_test)

             i      k      u
330603   False   True  False
1088698  False   True  False
1082310  False   True  False
1003575  False   True  False
983519    True  False  False
...        ...    ...    ...
536848   False   True  False
541965   False   True  False
547529   False   True  False
666303    True  False  False
17624     True  False  False

[189808 rows x 3 columns]


In [67]:
print(y_pred[0])

[False  True False]


In [68]:
accuracy_score(y_test, y_pred)

0.9656653038860322

In [69]:
classification_report(y_test, y_pred, target_names=['i', 'k', 'u'])

/home/selloh/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/selloh/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/selloh/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'              precision    recall  f1-score   support\n\n           i       0.97      0.83      0.90     25876\n           k       0.98      0.99      0.99    159907\n           u       0.98      0.80      0.88      4001\n\n   micro avg       0.98      0.97      0.97    189784\n   macro avg       0.98      0.87      0.92    189784\nweighted avg       0.98      0.97      0.97    189784\n samples avg       0.97      0.97      0.97    189784\n'

In [70]:
print(y_test)

             i      k      u
330603   False   True  False
1088698  False   True  False
1082310  False   True  False
1003575  False   True  False
983519    True  False  False
...        ...    ...    ...
536848   False   True  False
541965   False   True  False
547529   False   True  False
666303    True  False  False
17624     True  False  False

[189808 rows x 3 columns]
